## Solving Systems using Elimination


In this section we demonstrate all of the code needed to solve the linear system $AX=B$ using elimination.  We will restrict our objective to the case that $A$ is a square $n\times n$ matrix and consider the case where $A$ is $m\times n$ at a later point.

When writing code to perform a complex problem, it is often a good idea to first break up the task, and write code to carry out smaller pieces.  Once we have code to reliably perform the small tasks, we can assemble the pieces to solve the larger problem.  In our case we will break down the solution method into two parts.

1. Carry out elimination on the associated augmented matrix.
2. Perform back substitution on the triangular system that elimination produces.

It is also beneficial to consider how we might write the code now so that it will it can reuse it for other tasks later. 

In [2]:
import numpy as np
import laguide as lag

### Back Substitution routine

We will start with the back substitution step, since that is the easier part.  If the elimination step is successful, we will have an upper triangular system $UX=B$ that has the following form.

$$
\begin{equation}
\left[ \begin{array}{rrrr} * & * & * & * \\ 0 & * & * & * \\ 0 & 0 & * & * \\ 0 & 0 & 0 & * \end{array}\right]
\left[ \begin{array}{r}  x_1 \\  x_2  \\ x_3 \\ x_4  \end{array}\right]=
\left[ \begin{array}{r}  * \\  *  \\ * \\ *  \end{array}\right]
\end{equation}
$$

We will put the code in a function so that that it is easy to reuse later.  For this function, let's suppose that we are given the upper triangular matrix $U$ and the known vector $B$ and we want to find the vector $X$ so that $UX=B$.  Note we could make other assumptions, such as the matrix $U$ having diagonal entries equal to 1, but if we make fewer assumptions, the code will be more useful later.

In [7]:
def BackSubstitution(U,B):
# =============================================================================
#     U is a NumPy array that represents an upper triangular square mxm matrix.  
#     B is a NumPy array that represents an mx1 vector     
#     BackSubstitution will return an mx1 vector that is the solution of the
#     system UX=B.
# =============================================================================
    m = U.shape[0]  # m is number of rows and columns in U
    X = np.zeros((m,1))
    
    for i in range(m-1,-1,-1):  # Calculate entries of X backward from m-1 to 0
        X[i] = B[i]
        for j in range(i+1,m):
            X[i] -= U[i][j]*X[j]
        X[i] /= U[i][i]
    return X

Before moving on, let's test this function.  We can build a matrix with the proper triangular form, *choose a solution$, and then construct a system $UX=B$ so that we know the solution.

In [8]:
# Make up an upper triangular matrix.  Could we make a random upper triangular matrix here?  What could go wrong?
U = np.array([[3,0,1],[0,1,-1],[0,0,-3]])  
# We will choose the solution X_true.  We can put in any numbers we like here. 
X_true = np.array([[2],[4],[6]])
B = U@X_true
# Call the function.  It should produce the same array as X_true if it works correctly.
X = BackSubstitution(U,B)

print(X)

[[2.]
 [4.]
 [6.]]


### Row Reduction routine

Elimination is the larger and more complex part of the solution method.  It is also a common task that will arise in future sections, so we will want some code that we can reuse at a later point.  We want a function that will carry out all the steps of elimination, and just return the end result.  It is not necessary to see all the inividual row operations that took place in order solve the problem.  Ideally, we would like the function to carry out the elimination on arrays of any size or shape, and also be able to _make the decision_ to perform row swaps when necessary.

To clarify the goal, the function should accept an arbitrary array and produce an array that has the following properties.

1. The entries in the pivot positions are 1.
2. The entries below the pivot postions are 0.

Such a matrix is said to be in a **row echelon form**.  Here are two examples of matrices in the form that we seek.

   
$$
\begin{equation}
\left[ \begin{array}{cccc} 1 & * & * & * \\ 0 & 1 & * & * \\ 0 & 0 & 1 & * \end{array}\right]
\end{equation}
$$


$$
\begin{equation}
\left[ \begin{array}{cccccc} 1 & * & * & * & * & * \\ 0 & 0 & 1 & * & * & * \\ 0 & 0 & 0 & 1 & * & * \end{array}\right]
\end{equation}
$$

*Note that the requirement of having a 1 in the pivot position is not strictly necessary, and is indeed not necessary to make use of our back substitution routine.*

In [11]:
def RowReduction(A):
    ''' 
    RowReduction performs steps of elimination with no pivot strategy and
    returns a row echelon form of the matrix A.
    
    Parameters
    ----------
    A : NumPy array object of dimension mxn
    
    Returns
    -------
    B: NumPy array object of dimension mxn
    '''
    
    m = A.shape[0]  # m is number of rows in A
    n = A.shape[1]  # n is number of columns in A

    B = np.zeros((m,n))
    for i in range(m):
        for j in range(n):
            B[i][j] = A[i][j]

    if m < n:
        elimination_steps = m
    else:
        elimination_steps = n

    # For each step of elimination, we find a suitable pivot, move it into
    # position and create zeros for all entries below.
    
    for k in range(elimination_steps):
        # Set pivot as (k,k) entry
        pivot = B[k][k]
        pivot_row = k
        
        # Find a suitable pivot if the (k,k) entry is zero
        while(pivot == 0 and pivot_row < m):
            pivot_row += 1
            pivot = B[pivot_row][k]
            
        # Swap row if needed
        if (pivot_row != k):
            B = lag.RowSwap(B,k,pivot_row)
            
        # If pivot is nonzero, carry on with elimination in column k
        if (pivot != 0):
            B = lag.RowScale(B,k,1./B[k][k])
            for i in range(k+1,m):    
                B = lag.RowAdd(B,k,i,-B[i][k])
    return B
    

    

Note that in this routine we make use of the row operations written earlier.  Since those functions are note written in *this notebook*, we need to import them from the $\texttt{laguide}$ module.

Let's test the routine on a random array.  Run the code on several random matrices of different sizes and shapes.  Does it always work?  Do you notice any unusual results?  Does it depend on the size or shape?  Does it depend on the range of numbers used?

In [12]:
R = np.random.randint(-8,8,size=(4,4))
print(R)
print('\n')
print(RowReduction(R))

[[ 6  3 -4  5]
 [ 5 -2  3 -1]
 [ 3  7  0  4]
 [ 2 -4  7 -4]]


[[ 1.          0.5        -0.66666667  0.83333333]
 [-0.          1.         -1.40740741  1.14814815]
 [ 0.          0.          1.         -0.49429658]
 [ 0.          0.          0.          1.        ]]


If you run this test enough times, you are likely to come across an example where the results look a little different.  Here is one such case.

In [13]:
NumericalReductionExample=np.array([[7,-6,6,-8],[-3,-5,-7,2],[1,-4,-7,-6],[-1,0,-2,-8]])
print(RowReduction(NumericalReductionExample))

[[ 1.00000000e+00 -8.57142857e-01  8.57142857e-01 -1.14285714e+00]
 [-0.00000000e+00  1.00000000e+00  5.84905660e-01  1.88679245e-01]
 [-0.00000000e+00 -0.00000000e+00  1.00000000e+00  7.08463950e-01]
 [-0.00000000e+00 -0.00000000e+00  1.30206303e-17  1.00000000e+00]]


There are two things that we observe in this example.  First and most obvious is that the entries are all displayed in scientific notation.  The more disturbing observation is that the result is not exactly what we wanted.  The elimination process is supposed to produce zeros for all the entries below the main diagonal, but in this case there is one entry that is not $\texttt{0.000}$.  Instead it is an extremely small number, close to $10^{-17}$.  At this point we might question the code and start looking for errors, but the problem here does not lie with the code.  The issue here is something deeper and involves the precision limitations of the computer.  When the computer carries out these calculations it does not always work with exact numbers.  In most cases the results get rounded off to a number that the machine can represent.  This limitation is known as **roundoff error** and it is the reason we do not get exactly zero for all of the entries below the diagonal.  

Roundoff error can present a significant challenge if we work with large arrays, and the errors are allowed to accumulate and compound.  There are strategies that can be employed to mitigate this error, and we will revisit this dicussion in a later section.  For now we will carry on with elimination with the awareness that the results we get are not always *exactly correct*. 

#### Exercises

- Try out the code on two different arrays that require the use of $\texttt{RowSwap}$.
- Is it possible to test random arrays that require the use of $\texttt{RowSwap}$?

### System Solve routine

Now we can combine the $\texttt{RowReduction}$ and the $\texttt{BackSubstitution}$ routines together to carry out the solution algorithm for the system $AX=B$.  Since we will be the users of this function, let's try to make it easy to use.  One simple possibility is that the user of the function will supply $A$ and $B$, and the function will return the solution $X$.  Let's list the steps that need to be completed.

1. Build an augmented matrix.
2. Apply $\texttt{RowReduction}$.
3. Split the matrix.
4. Apply $\texttt{BackSubstitution}$ and return the result.

Note that there are other ways we could build our routine.  We could require the user to supply the augmented matrix for example, but then that means the user (us) has to do step 1 everytime they use this routine.  It is better to let the function handle that step.

In [14]:
def SolveSystem(A,B):
    ''' 
    SystemSolve computes the solution to AX=B by elimination in the case that
    A is a square nxn matrix
    
    Parameters
    ----------
    A : NumPy array object of dimension nxn
    B : NumPy array object of dimension nx1
    
    Returns
    -------
    X: NumPy array object of dimension nx1
    '''
    # Check shape of A
    if (A.shape[0] != A.shape[1]):
        print("SolveSystem accepts only square arrays.")
        return
    n = A.shape[0]  # n is number of rows and columns in A

    # 1. Join A and B to make the augmented matrix
    A_augmented = np.hstack((A,B))
    
    # 2. Carry out elimination    
    R = RowReduction(A_augmented)

    # 3. Split R back to nxn piece and nx1 piece
    B_reduced = R[:,n:n+1]
    A_reduced = R[:,0:n]

    # 4. Do back substitution
    X = BackSubstitution(A_reduced,B_reduced)
    return X

Let's test the routine by building a matrix, choosing a solution, and constructing a system $AX=B$ so that we know the solution.

In [15]:
A = np.array([[1,2,3],[0,1,-2],[3,3,-2]])
# We will choose the solution X_true
X_true = np.array([[1],[1],[1]])
# Now make B so that the solution to AX=B is X_true
B = A@X_true
X = SolveSystem(A,B)
print(X)

[[1.]
 [1.]
 [1.]]


Next, we modify a couple of lines to produce a completely random system with random solution.  We will use $\texttt{SolveSystem}$ to find the solution and then compute the difference between the result and the actual known solution.

In [16]:
A = np.random.randint(-8,8,size=(4,4))
# We set X_true for a random solution
X_true = np.random.randint(-8,8,size=(4,1))
# Now make B so that the solution to AX=B is X_true
B = A@X_true
X = SolveSystem(A,B)
# Print the difference in computed solution and actual solution
print(X_true-X)

[[ 1.77635684e-15]
 [ 0.00000000e+00]
 [-1.77635684e-15]
 [ 0.00000000e+00]]


#### Exercise

- Set up a similar test using $\text{np.random.rand}$ to get random floats as matrix entries.

In [17]:
A = np.random.rand(4,4)
# We set X_true for a random solution
X_true = np.random.rand(4,1)
# Now make B so that the solution to AX=B is X_true
B = A@X_true
X = SolveSystem(A,B)
# Print the difference in computed solution and actual solution
print(X_true-X)

[[ 1.11022302e-16]
 [-8.88178420e-16]
 [ 2.49800181e-15]
 [-3.21964677e-15]]


#### Exercises

- Modify $\texttt{RowReduction}$ to compute what is known as the **reduced row echelon form**.  Name the new function $\texttt{RREF}$.  
    A matrix in reduced row echelon form should have the following properties.
    - The entries in pivot positions are 1.
    - The entries below each pivot position are 0.
    - The entries *above* each pivot position are 0.
   Here are some examples of matrices that are in reduced row echelon form.
   
   
$$
\begin{equation}
\left[ \begin{array}{cccc} 1 & 0 & 0 & * \\ 0 & 1 & 0 & * \\ 0 & 0 & 1 & * \end{array}\right]
\end{equation}
$$


$$
\begin{equation}
\left[ \begin{array}{cccccc} 1 & * & 0 & 0 & * & * \\ 0 & 0 & 1 & 0 & * & * \\ 0 & 0 & 0 & 1 & * & * \end{array}\right]
\end{equation}
$$
    Note that for the system represented by the augmented matrix in the first example, the solution is given by the entries in the final column.  There is no need for back substitution if the augmented matrix is in reduced row echelon form.

- Test your $\text{RREF}$ on random matrices with different shapes including those above ($3\times 4$ and $3 \times 6$)
- Construct a $3 \times 3$ system with a known solution and compare the solutions produced using $\texttt{SolveSystem}$ with those produced using $\texttt{RREF}$. 